In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

# path = get_file(
#     'text.txt',
#     origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open('textrus.txt') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

corpus length: 549559
total chars: 90
nb sequences: 183173
Vectorization...
Build model...
Epoch 1/20
1431/1432 [============================>.] - ETA: 0s - loss: 2.2520
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ам светит только тюрьма… или смерть. я п"
ам светит только тюрьма… или смерть. я провой промот в сем он своим своим не своим своим не он не поставила своим не он от не своим в своим своим приками не возмого своим своим в сорого своим в солдат своим в своим своим на своим лице на смогли обравил свои примом не он не своим своим своим своим своим от своим прика собраться своим своим прика солдата своим с грейсон не оправил своим был своим в ского концори своим прикат он в солдат с
----- diversity: 0.5
----- Generating with seed: "ам светит только тюрьма… или смерть. я п"
ам светит только тюрьма… или смерть. я прико ми не своим замот. прокатался на оружения обнера, но слашином концорит сохдата мехов протой звездую улице двкахов не своим 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: RuntimeWarning: divide by zero encountered in log


 огней не подошел они свистов рассчитывали и все не могли бы оставился на контролица лазерными меха. тогда тогда не слышал они полагался так и следи св
----- diversity: 1.0
----- Generating with seed: " грейсон следил, как мех ворочался, высл"
 грейсон следил, как мех ворочался, высложненного «shadow-hawk’а», в их та несколько слабы, чуние огруженный этого и контротка руки, будет повернулся на километрой, тем это тягались рилезким шлисом.
 — цепь со скреы ему… дворец, ополчения вышагались человеку спушка раяплели через стартовой увиделающего отсюка плать возфбились на ремонтный голод!
 — у них нам — с решилось за нескольку единстванное грейсона.
 шум точки, разя-будлованное о
----- diversity: 1.2
----- Generating with seed: " грейсон следил, как мех ворочался, высл"
 грейсон следил, как мех ворочался, выслучился за рука, жалов его это. — грейсон и япило, зажим, гергикаций, чтобы чутья, ее внутренние, несождния точки напредился иа-то ничего тему
 они есло небвазожить нос! от том.
 приби